In [ ]:
import mlflow
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

from hyperopt import fmin, tpe, Trials, STATUS_OK, hp, space_eval
from functools import partial

from pickle import dump

from scripts.Preprocessing import Preprocessing
from scripts.BinaryClassificationTraining import BinaryClassificationTraining

from scripts.config import (year_month_train, 
    input_data_path_train,
    seed)

In [ ]:
local_path_save = './local_artifacts_tmp/08_SVM/'
year_month = year_month_train
input_data_path = input_data_path_train

### MLFlow setting

In [ ]:
if not os.path.exists(local_path_save):
    os.makedirs(local_path_save)

#save all metadata in a sqlite db. Artifacts will be saved on local folder ./mlflow    
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Name of the experiment
exp_name = "06 - SVM"
# set up MlFlow axperiment
experiment_id = mlflow.set_experiment(exp_name)

### Experiments

In [ ]:
run_name = 'base'

In [ ]:
svm_training = BinaryClassificationTraining(
    input_data_path,
    local_path_save,
    year_month,
    'svm')

In [ ]:
prepr = Preprocessing(input_data_path_train, task_type='classification')
X, Y = prepr.read_dataframe(request_tgt=True)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=seed)


#preprocessing ohe
shapes_pre = (X_train.shape[0], X_test.shape[0])
X_train_ohe, ohe, scaler = prepr.preprocess_for_classification(df=X_train, fit_ohe=True, perform_scaling=True)
X_test_ohe, _, _ = prepr.preprocess_for_classification(df=X_test, 
                                                    fit_ohe=False, 
                                                    ohe=ohe,
                                                    perform_scaling=True,
                                                    scaler=scaler)
assert shapes_pre == (X_train.shape[0], X_test.shape[0])
dump(ohe, open(local_path_save + run_name + '_ohe.pkl', 'wb'))
dump(scaler, open(local_path_save + run_name + '_scaler.pkl', 'wb'))

In [ ]:
# Calculating the baseline
svm_training.calculate_classification_baseline(Y_train=Y_train, Y_test=Y_test, run_name=run_name)

In [ ]:
# TODO complete the experiment using your implementation from BinaryClassificationTraining